In [1]:
import re
import csv
from math import log, exp   # 기본 자연로그
from collections import Counter

In [80]:
def start():
    train_datas = open_csv() 
    
    test_data = input()
    prob = naive_bayes(train_datas, test_data, 0.5, 0.5)
    
    print(f"리뷰가 부정적인 확률 : {round(prob[0], 5)}, 긍정적일 확률 : {round(prob[1], 5)}")

    
def open_csv():
    
    train_datas = [[], []]
    f_train = open("./data/ratings.txt", 'r', encoding='utf-8')
    csvreader = csv.reader(f_train, delimiter='\t')
    next(csvreader)
    for line in csvreader:
        if line[2] == '1':
            train_datas[1].append(line[1])
        else:
            train_datas[0].append(line[1])

            
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])]

   
#----------------------------------------

def start_validate():
    train_datas, test_datas = open_csv_validate()
    
    prob_neg = naive_bayes(train_datas, test_datas[0], 0.5, 0.5)
    prob_pos = naive_bayes(train_datas, test_datas[1], 0.5, 0.5)
    
    print(f"부정 텍스트가 부정적일 확률 : {round(prob_neg[0], 5)}, 긍정적인 확률 : {round(prob_neg[1], 5)}")
    print(f"긍정 텍스트가 부정적일 확률 : {round(prob_pos[0], 5)}, 긍정적인 확률 : {round(prob_pos[1], 5)}")
    

    

def open_csv_validate():
    
    train_datas = [[], []]
    f_train = open("./data/ratings_train.txt", 'r', encoding='utf-8')
    csvreader = csv.reader(f_train, delimiter='\t')
    next(csvreader)
    for line in csvreader:
        if line[2] == '1':
            train_datas[1].append(line[1])
        else:
            train_datas[0].append(line[1])
            
    test_datas = [[], []]
    f_test = open("./data/ratings_test.txt", 'r', encoding='utf-8')
    csvreader = csv.reader(f_test, delimiter='\t')
    next(csvreader)
    for line in csvreader:
        if line[2] == '1':
            test_datas[1].append(line[1])
        else:
            test_datas[0].append(line[1])
            
    return [' '.join(train_datas[0]), ' '.join(train_datas[1])], [' '.join(test_datas[0]), ' '.join(test_datas[1])]



def calculate_doc_prob(train_data, test_data, nowords_weight):
    log_prob = 0 
        
   
    sw_train_data = re.compile("[^\w]").sub(' ', train_data)

    sw_train_token = sw_train_data.split() 
    train_vector = dict(Counter(sw_train_token))
    

    sw_test_data = re.compile("[^\w]").sub(' ', test_data)
    sw_test_token = sw_test_data.split()
    test_vector = dict(Counter(sw_test_token)) 
    
   
    total_wc = len(sw_train_token)
    
    for word in test_vector:
        if word in train_vector:
            log_prob += log(train_vector[word]/total_wc)
        else:
            log_prob += log(nowords_weight/total_wc) 
            
    return log_prob



def naive_bayes(train_datas, test_data, pos_prob, neg_prob):
    test_pos_prob = calculate_doc_prob(train_datas[1], test_data, 0.1) + log(pos_prob) 
    test_neg_prob = calculate_doc_prob(train_datas[0], test_data, 0.1) + log(neg_prob)
     
    maxprob = max(test_neg_prob, test_pos_prob)
    test_pos_prob -= maxprob
    test_neg_prob -= maxprob
    print(test_pos_prob, test_neg_prob)
    test_pos_prob = exp(test_pos_prob)
    test_neg_prob = exp(test_neg_prob)
    normalized_prob = [test_neg_prob/(test_neg_prob+test_pos_prob), test_pos_prob/(test_neg_prob+test_pos_prob)]
    
    return normalized_prob



    


In [81]:
start()

나이스
0.0 -2.226970516162611
리뷰가 부정적인 확률 : 0.09735, 긍정적일 확률 : 0.90265


In [82]:
start_validate()

-22891.8269452611 0.0
0.0 -19819.128885605838
부정 텍스트가 부정적일 확률 : 1.0, 긍정적인 확률 : 0.0
긍정 텍스트가 부정적일 확률 : 0.0, 긍정적인 확률 : 1.0
